In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# uploading shooting data

shootings = pd.read_csv('https://data.cityofnewyork.us/resource/833y-fsy8.csv?$limit=9999999999999999')

shootings['occur_date'] = pd.to_datetime(shootings['occur_date']) # converting to datetime
shootings_06_12 = shootings[shootings['occur_date'] < pd.to_datetime('2013-01-01')] # filtering out dates after 2012

In [4]:
# grouping by year and precinct

shootings_06_12 = shootings_06_12.groupby(['precinct', shootings_06_12['occur_date'].dt.to_period('Y')]).count()[['incident_key']]
shootings_06_12 = shootings_06_12.rename(columns={'incident_key':'total_shootings'}) # changing column title

In [5]:
# the 2010 population breakdown within each precinct as they are drawn in June 2020
# source: https://johnkeefe.net/nyc-police-precinct-and-census-data

precinct_pop_data_10 = pd.read_csv('../data/input/nyc_2010pop_2020precincts.csv')

In [6]:
# seeing if any precincts don't align between datasets

set(precinct_pop_data_10['precinct_2020'].to_list()) ^ set(shootings_06_12.index.get_level_values('precinct').to_list())

{22, 121}

In [7]:
# removing missing precincts

precinct_pop_data_10 = precinct_pop_data_10[(precinct_pop_data_10['precinct_2020'] != 22) & (precinct_pop_data_10['precinct_2020'] != 121)][['precinct_2020', 'P0010001']]
precinct_pop_data_10 = precinct_pop_data_10.set_index('precinct_2020')

# adding population per precinct (based on 2010 census)
# 'P0010001' is total population according to https://www.documentcloud.org/documents/87708-pl94-171-1.html#document/p64
# using pop_dict to pair each precinct to total pop, then matching with shootings_06_12['precinct'] to add pop to shootings_06_12['2010_population']  

pop_dict = pd.Series(precinct_pop_data_10['P0010001'].values,index=precinct_pop_data_10.index.astype(str)).to_dict()
shootings_06_12 = shootings_06_12.reset_index(level=['precinct'])
shootings_06_12['precinct'] = shootings_06_12['precinct'].astype(str)
shootings_06_12['2010_population'] = shootings_06_12['precinct'].astype(str).str.findall('|'.join([fr'\b{w}\b' for w in pop_dict.keys()])).apply(", ".join).map(pop_dict)

# creating shootings per 100K column using 'total_shootings' and '2010_population'

shootings_06_12['shootings_per_100K'] = (shootings_06_12['total_shootings'] / shootings_06_12['2010_population']) * 100000
shootings_06_12

,precinct,total_shootings,2010_population,shootings_per_100K
occur_date,,,,
2007,1,1,66679,1.499723
2008,1,1,66679,1.499723
2009,1,1,66679,1.499723
2010,1,8,66679,11.997780
2012,1,2,66679,2.999445
...,...,...,...,...
2007,123,3,98032,3.060225
2008,123,2,98032,2.040150
2009,123,2,98032,2.040150


In [8]:
# taking the mean of the annual shooting rate (2006-2012)

shootings_06_12 = shootings_06_12.groupby('precinct').mean()[['shootings_per_100K']] # grouping by precinct
shootings_06_12.index = shootings_06_12.index.astype(int)
shootings_06_12 = shootings_06_12.sort_index()

In [10]:
# downloading results

# shootings_06_12.to_csv('../data/output/NYC-precinct-shootings_mean_2006_2012.csv')